In [3]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Gemma2ForCausalLM
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.schema.output_parser import StrOutputParser
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever
from peft import PeftModel

In [4]:
from utils.extract_module import daconCustomExtractor

def process_pdf(file_path):
    # 1. pdf별 page 추출
    doc = fitz.open(file_path)
    outpt_dir = "/home/a2024712006/dacon/extract_image"
    chunk_list = []
    for page_number in range(doc.page_count):
        page = doc.load_page(page_number)
        
        #2. page별 테이블 추출 
        tables = page.find_tables()
        raw_text_list = []
        for table in tables:
            # 테이블을 감싸는 영역 계산
            min_x, min_y = float('inf'), float('inf')
            max_x, max_y = float('-inf'), float('-inf')
            
            for cell in table.cells:
                x0, y0, x1, y1 = cell[:4]  # 셀 좌표 추출
                min_x = min(min_x, x0)
                min_y = min(min_y, y0)
                max_x = max(max_x, x1)
                max_y = max(max_y, y1)
            # 1) 발견된 테이블 영역 
            table_rect = fitz.Rect(min_x, min_y, max_x, max_y)
            # 2) 발견된 테이블 영역의 테이블 형식 텍스트
            table_text = "\n"
            for row in table.extract():
                table_text += str(row)
                table_text += "\n"
            # 3) 발견된 테이블 영역의 날 것 텍스트 
            clipped_text = page.get_text("text", clip=table_rect)
            
            # 4) 날 것 텍스트 => 테이블 형식 텍스트로 변환
            raw_text_list.append((clipped_text, table_text))
            
        # 2. page별 이미지 추출        
        extractor = daconCustomExtractor(page) 
        bboxes = extractor.detect_svg_contours(page_number+1, output_dir=outpt_dir, min_svg_gap_dx=25.0, min_svg_gap_dy=25.0, min_w=2.0, min_h=2.0)

        # # 텍스트를 chunk로 분할
        # splitter = RecursiveCharacterTextSplitter(
        #     chunk_size=512,
        #     chunk_overlap=32
        # )    
        # 3. 이미지별 텍스트 추출
        for i, bbox in enumerate(bboxes):
            x0, y0, x1, y1 = bbox 
            
            full_text = page.get_text("text", clip=fitz.Rect(x0, y0, x1, y1))
            for clipped_text, table_text in raw_text_list:
                if clipped_text in full_text:
                    full_text = full_text.replace(clipped_text, table_text)
                    
            chunk_list.append(full_text)
            # chunk_temp = splitter.split_text(full_text)
            # chunk_list.extend(chunk_temp)
            
    chunks = [Document(page_content=t) for t in chunk_list]
    return chunks

def create_vector_db(chunks, model_path="BAAI/bge-m3"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings, distance_strategy = DistanceStrategy.COSINE)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)

def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(chunks)
        faiss_retriever = db.as_retriever()
        # Retriever 생성
        retriever = EnsembleRetriever(
            retrievers=[kiwi_bm25_retriever, faiss_retriever],
            weights=[0.5, 0.5],
            search_type="mmr",
        )
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases

In [5]:
base_directory = '/home/a2024712006/dacon' # Your Base Directory
df = pd.read_csv('/home/a2024712006/dacon/test.csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


/home/a2024712006/miniconda3/envs/dacon/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Processing PDFs:  11%|█         | 1/9 [00:14<01:53, 14.23s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


Processing PDFs:  22%|██▏       | 2/9 [00:25<01:28, 12.59s/it]

...

Processing PDFs:  78%|███████▊  | 7/9 [03:06<01:13, 36.67s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [03:53<00:39, 39.66s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [04:35<00:00, 30.63s/it]


In [ ]:
import transformers
import torch


model_id = "rtzr/ko-gemma-2-9b-it" 

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

pipeline.model.eval()

In [10]:
# return: final_context, full_text
# final_context: 최종 llm에게 넘길 context
# full_doc_text: reranking하기 전 context
def llm_reranker(documents, question):
    context = []
    full_doc_text = ""
    for i in range(len(documents)):
        if i > 4:
            break
        context.append(documents[i].page_content)
        if i != 0:
            text = f"문서{i}:" + "\n\n" + documents[i].page_content
            full_doc_text += f"\n\n{text}\n==="

    prompt = f"""
    정보: 
        {full_doc_text}

    위 정보에서 아래 질문에 대한 답을 유추할 수 있고 가장 유사한 문서를 2개 골라서 문서의 번호만 출력해줘

    질문: {question}

    """
    messages = [
    {"role": "user", "content": f"{prompt}"}
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<end_of_turn>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=2048,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0]["generated_text"][len(prompt):]
    print(f"Answer: 0, {response}\n")
    
    reranked_idx_list = [0]
    for char in response:
        if char.isdigit() and char != "0":
            idx = int(char)
            if idx not in reranked_idx_list and idx < 5:
                reranked_idx_list.append(idx)
    
    final_context = ""
    for idx in reranked_idx_list:
        final_context += f"\n{context[idx]}\n"
    return final_context, full_doc_text

# return: response
def llm_answer(context, question):
    prompt = f"""
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문: {question}
    
    주어진 질문에만 답변하세요. 문장으로 답변해주세요. 답변할 때 질문의 주어를 써주세요.
    답변:
    """
    messages = [
    {"role": "user", "content": f"{prompt}"}
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<end_of_turn>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=2048,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0]["generated_text"][len(prompt):]
    return response

In [11]:
# 정규화된 키로 데이터베이스 검색
normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}

debug_context = {}
debug_full_doc_text ={}

results = []
# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']
    retriever = normalized_keys[source]['retriever']
    documents = retriever.get_relevant_documents(question)

    context, full_doc_text = llm_reranker(documents, question)
    debug_context['question'] = context 
    debug_full_doc_text['question'] = full_doc_text
    
    print(f"Question: {question}")
    response = llm_answer(context, question)
    print(f"Answer: {response}\n")
    
    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": response
    })

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]

Answer: 0, 1, 3 


Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Answering Questions:   1%|          | 1/98 [00:04<07:25,  4.59s/it]

Answer: 2022년 혁신창업사업화자금(융자)의 예산은 2,300 백만원 입니다.  


Answer: 0, 1, 2 


Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


Answering Questions:   2%|▏         | 2/98 [00:15<12:57,  8.09s/it]

...

Answer: 국가결산보고서는 국가회계기준에 따라 우발부채, 충당부채, 우발자산을 정의하고, 이를 주석에 공시하며, 지방자치단체 회계기준은 우발상황, 우발손실, 우발이익을 정의하며, 우발부채는 발생 가능성과 금액의 추정 가능성에 따라 재무제표상 재정상태표에 인식될 수 있습니다. 


Answer: 0, 01, 03 




Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?


Answering Questions:  99%|█████████▉| 97/98 [13:01<00:07,  7.74s/it]

Answer: 우발부채는 미래에 특정 사건이 발생하지 않는 한 발생하지 않는 잠재적 의무 또는 부채를 말합니다.  그 관리가 중요한 이유는 미래의 다양한 의제의무를 포괄하고 재정위험을 발생시킬 수 있기 때문입니다.  


Answer: 0, 18, 20  


Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?


Answering Questions: 100%|██████████| 98/98 [13:18<00:00,  8.15s/it]

Answer: 보증은 규정된 조건 발생 시 경제적 가치의 지급을 요구하는 법적 또는 계약에 의한 금융 협정입니다.  파생금융상품 형태의 보증과 표준화 보증은 재정상태표에 부채로 기록됩니다. 표준화 보증은 통상 아주 적은 금액에 대해 획일적 조건으로 대규모로 발행되는 보증이며, 수출신용 보증, 환보증, 다양한 종류의 보험, 농민융자, 모기지론, 학자금융자, 중소기업융자 등이 그 예시입니다. 이러한 보증들은 주로 유사한 특징을 지니고 위험 분산을 목적으로 발행됩니다. 






In [12]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("./sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in documents]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./8_22_submission.csv", encoding='UTF-8-sig', index=False)



TypeError: 'Document' object is not subscriptable